In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import itertools
import networkx as nx
import ast
import math

In [2]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return (sum_x/length, sum_y/length, sum_z/length)

In [3]:
def split(word):
    return [char for char in word]

In [4]:
def getDistances(list1):
    distances_list1 = {}
    for p1 in list1:
        index_str = '[' + str(p1[0]) +','+ str(p1[1]) +','+ str(p1[2]) + ']'
        distances_list1[index_str] = []
        for p2 in list1:
            dist = math.dist(p1,p2)
            if dist != 0:
                distances_list1[index_str].append(math.dist(p1,p2))
    return distances_list1

In [20]:
def getDistances_2(list1):
    distances_list1 = {}
    for p1 in list1:
        index_str = '[' + str(p1[0]) +','+ str(p1[1]) +','+ str(p1[2]) + ']'
        distances_list1[index_str] = []
        for p2 in list1:
            dist = math.dist(p1,p2)
            if dist != 0:
                index_str_2 = '[' + str(p2[0]) +','+ str(p2[1]) +','+ str(p2[2]) + ']'
                distances_list1[index_str].append((index_str_2,dist))
    return distances_list1

In [5]:
def getOverlap(list1,list2):
    list1 = getDistances(list1)
    list2 = getDistances(list2)
    result = []
    xo = []
    for d1 in list1:
        maxOverlap = 0
        maxOverlapIndex = None 
        for d2 in list2:
            setA = set(list1[d1])
            setB = set(list2[d2])
            overlap = setA & setB
            result1 = float(len(overlap)) / len(setA) * 100
            if result1 > maxOverlap:
                maxOverlap = result1 
                maxOverlapIndex = d2
        result.append((d1,maxOverlapIndex,maxOverlap))
        xo.append(maxOverlap)
    result.sort(key=lambda i:i[2],reverse=True)    
    cnt = 0
    score = result[0][2]
    for ele in result:
        if ele[2] == score:
            cnt += 1
    c_items = Counter(xo).items()
    print(Counter(xo),c_items)
    for c in list(c_items)[1:]:
        if c[1] == 12:
            return result[:12]
    return -1

In [57]:
getOverlap_2(scanners[0],scanners[1])

[('[528,-643,409]', '[-460,603,-452]'),
 ('[390,-675,-793]', '[-322,571,750]'),
 ('[-537,-823,-458]', '[605,423,415]'),
 ('[-485,-357,347]', '[553,889,-390]'),
 ('[-345,-311,381]', '[413,935,-424]'),
 ('[-661,-816,-575]', '[729,430,532]'),
 ('[-618,-824,-621]', '[686,422,578]'),
 ('[-447,-329,318]', '[515,917,-361]'),
 ('[544,-627,-890]', '[-476,619,847]'),
 ('[423,-701,434]', '[-355,545,-477]'),
 ('[459,-707,401]', '[-391,539,-444]'),
 ('[404,-588,-901]', '[-336,658,858]')]

In [66]:
def getOverlap_2(list1,list2):
    list1 = getDistances_2(list1)
    list2 = getDistances_2(list2)
    result = []
    for d1 in list1:
        cnt = 0
        for d2 in list2:
            cnt = 0
            result = []
            for i in list1[d1]:
                for k in list2[d2]:
                    if i[1] == k[1]:
                        cnt += 1
                        result.append((i[0],k[0]))
            if cnt >= 11:
                result.append((d1,d2))
                return result 
        if cnt >= 11:
            return result 
    return -1

In [58]:
def getOrigin(list1,list2):
    overlap = getOverlap_2((list1),(list2))
    print(overlap)
    overScan1 = [ast.literal_eval(x[0]) for x in overlap]
    overScan2 = [ast.literal_eval(x[1]) for x in overlap]
    center1 = centeroidnp(np.array(overScan1))
    center2 = centeroidnp(np.array(overScan2))
    #print(center1,center2)
    for ele in overScan1:
        ele[0] -= center1[0]
        ele[1] -= center1[1]
        ele[2] -= center1[2]
    for ele in overScan2:
        ele[0] -= center2[0]
        ele[1] -= center2[1]
        ele[2] -= center2[2]    

    P = np.matrix([[x[0] for x in overScan1],[x[1] for x in overScan1],[x[2] for x in overScan1]]).T
    Q = np.matrix([[x[0] for x in overScan2],[x[1] for x in overScan2],[x[2] for x in overScan2]]).T
    #print(P.shape,Q.shape)
    H = P.T * Q 
    u, s, vh = np.linalg.svd(H, full_matrices=True)
    v = vh.T
    d = np.linalg.det(v @ u.T)
    e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
    r = v @ e @ u.T
    t = ast.literal_eval(overlap[1][0]) - ast.literal_eval(overlap[1][1]) * r
    return ([0,0,0]*r+t,r,t)

In [61]:
def getOrigin_2(list1,list2):
    overScan1 = list1
    overScan2 = list2
    center1 = centeroidnp(np.array(overScan1))
    center2 = centeroidnp(np.array(overScan2))
    #print(center1,center2)
    for ele in overScan1:
        ele[0] -= center1[0]
        ele[1] -= center1[1]
        ele[2] -= center1[2]
    for ele in overScan2:
        ele[0] -= center2[0]
        ele[1] -= center2[1]
        ele[2] -= center2[2]    

    P = np.matrix([[x[0] for x in overScan1],[x[1] for x in overScan1],[x[2] for x in overScan1]]).T
    Q = np.matrix([[x[0] for x in overScan2],[x[1] for x in overScan2],[x[2] for x in overScan2]]).T
    #print(P.shape,Q.shape)
    H = P.T * Q 
    u, s, vh = np.linalg.svd(H, full_matrices=True)
    v = vh.T
    d = np.linalg.det(v @ u.T)
    e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
    r = v @ e @ u.T
    t = ast.literal_eval(overlap[1][0]) - ast.literal_eval(overlap[1][1]) * r
    return ([0,0,0]*r+t,r,t)

In [69]:
# Using readlines()
file1 = open('aoc-19-ex', 'r')
file1 = open('aoc-19', 'r')
Lines = file1.readlines()
Lines = [x.replace('\n','') for x in Lines]
scanners = []
helper = []
for ele in Lines:
    if ele.find('scanner') > 0 :
        scanners.append(helper)
        helper = []
    else:
        xo = ele.split(',')
        helper.append([int(xo[0]),int(xo[1]),int(xo[2])])
scanners.append(helper)
scanners.pop(0)

[]

In [70]:
# get scanner ovelap
while len(scanners) > 1:
    scannerToMatch = None
    overlap = None 
    for s1 in scanners[1:]:
        overlap = getOverlap_2(scanners[0],s1)
        if overlap != -1:
            print('scanner found',scanners.index(s1))#,overlap)
            scannerToMatch = s1.copy()
            scanners.remove(s1)
            break
    
    list1 = [ast.literal_eval(x[0]) for x in overlap]
    list2 = [ast.literal_eval(x[1]) for x in overlap]

    if scannerToMatch != None:
        p1,s1_r,s1_t = getOrigin_2(list1,list2)
        print('scanner at: ', [0,0,0]*s1_r+s1_t , len(scanners))
        helper = []
        for ele in scannerToMatch:
            xo = ele*s1_r+s1_t
            if not np.round(xo).tolist()[0] in scanners[0]:
                helper.append(np.round(xo).tolist()[0])
        scanners[0] += helper
print(len(scanners[0]))

scanner found 3
scanner at:  [[  77. -115. 1203.]] 30
scanner found 1
scanner at:  [[1242. -114. 1281.]] 29
scanner found 4
scanner at:  [[-1037.   -79.  1249.]] 28
scanner found 10
scanner at:  [[   21. -1353.  1367.]] 27
scanner found 5
scanner at:  [[  134. -2389.  1296.]] 26
scanner found 7
scanner at:  [[   85. -1257.  2529.]] 25
scanner found 8
scanner at:  [[  150.   -54. -1133.]] 24
scanner found 7
scanner at:  [[-1.60e+01 -1.00e+00 -2.29e+03]] 23
scanner found 5
scanner at:  [[   49. -1358. -2423.]] 22
scanner found 2
scanner at:  [[ 1312. -1365. -2268.]] 21
scanner found 8
scanner at:  [[ 1225. -2488.  1310.]] 20
scanner found 8
scanner at:  [[  72.  -11. 2511.]] 19
scanner found 3
scanner at:  [[  15. 1071. 2503.]] 18
scanner found 6
scanner at:  [[-2.000e+00 -5.800e+01  3.593e+03]] 17
scanner found 7
scanner at:  [[ 135. 1083. 1298.]] 16
scanner found 1
scanner at:  [[1260. 1127.  130.]] 15
scanner found 6
scanner at:  [[1286. 1210. 1317.]] 14
scanner found 1
scanner at:  [

In [75]:
scannersPositions = [[  77., -115.,1203.],
                    [1242., -114. ,1281.],
                    [-1037.,   -79. , 1249.],
                    [   21., -1353.,  1367.],
                    [  134., -2389.,  1296.],
                    [   85., -1257.,  2529.],
                    [  150.,   -54., -1133.],
                    #[-1.60e+01, -1.00e+00, -2.29e+03],
                    [   49., -1358., -2423.],
                    [ 1312., -1365., -2268.],
                    [ 1225., -2488.,  1310.],
                    [  72.,  -11., 2511.],
                    [  15., 1071., 2503.],
                    #[-2.000e+00, -5.800e+01,  3.593e+03],
                    [ 135., 1083., 1298.],
                    [1260., 1127.,  130.],
                    [1286., 1210., 1317.],
                    [2541., 1211., 1223.],
                    [  34., 1197.,  115.],
                    [ 168., 2290.,   96.],
                    #[ 1.000e+00,  3.502e+03, -2.700e+01],
                    [  79., 3526., 1210.],
                    [1311., 2340., 2468.],
                    [1337., 1099., 3663.],
                    [-1113.,   -98.,  2425.],
                    [-1160.,  -130.,  3771.],
                    [-2345.,     4.,  1359.],
                    [ 1203., -1191., -3577.],
                    [1264., 1056., 2377.],
                    [1235.,   -8.,   68.]]

In [76]:
maxDist = []
for s1 in scannersPositions:
    for s2 in scannersPositions:
        maxDist.append(abs(s1[0]-s2[0])+abs(s1[1]-s2[1])+abs(s1[2]-s2[2]))
print(max(maxDist))

10772.0


In [71]:
print(scanners[0])

[[136, -154, -15], [688, 487, 557], [806, -481, 565], [-566, 605, 547], [784, -477, 410], [-19, -78, 146], [772, -755, -623], [-551, 280, -333], [-672, -646, 864], [667, -777, -590], [-636, -891, -428], [796, -763, -662], [-552, 293, -594], [915, -495, 438], [711, 349, 666], [750, 485, 598], [591, 660, -656], [-583, -856, -458], [663, 780, -554], [-752, -510, 814], [710, 669, -530], [-566, -502, 862], [-478, -962, -447], [-669, 462, 554], [-506, 462, 559], [-536, 307, -473], [944.0, 585.0, 1798.0], [-229.0, 761.0, 2038.0], [626.0, -890.0, 1736.0], [-601.0, -885.0, 1650.0], [933.0, 565.0, 1871.0], [-247.0, 675.0, 1930.0], [620.0, -739.0, 1599.0], [-229.0, 739.0, 1815.0], [-561.0, -708.0, 1660.0], [-493.0, -735.0, 1662.0], [3.0, -129.0, 1219.0], [518.0, -889.0, 1598.0], [878.0, 436.0, 1788.0], [165.0, -10.0, 1298.0], [1331.0, -11.0, 1190.0], [2127.0, 477.0, 764.0], [2022.0, 534.0, 642.0], [1721.0, -898.0, 917.0], [1754.0, 254.0, 1889.0], [1776.0, 294.0, 1960.0], [1743.0, 243.0, 1815.0], 

In [ ]:
print(len(scanners[0]))